In [1]:
import pandas as pd
import google.generativeai as genai

In [4]:

GOOGLE_API_KEY=''
genai.configure(api_key=GOOGLE_API_KEY)

In [6]:
generation_config = {
  "candidate_count": 1,
  "max_output_tokens": 256,
  "temperature": 0.2
}

safety_settings=[
  {
    "category": "HARM_CATEGORY_DANGEROUS",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE",
  },
]
model = genai.GenerativeModel('gemini-pro', safety_settings=safety_settings, generation_config=generation_config)

In [7]:
response = model.generate_content("Give me python code to sort a list")
print(response.text)

```python
# Create a list of unsorted numbers
unsorted_list = [5, 2, 8, 3, 1, 9, 4, 7, 6]

# Sort the list in ascending order using the sorted() function
sorted_list = sorted(unsorted_list)

# Print the sorted list
print(sorted_list)
```


In [8]:
ds = pd.read_csv('release_train_patients')

In [14]:
ds

,AGE,DIFFERENTIAL_DIAGNOSIS,SEX,PATHOLOGY,EVIDENCES,INITIAL_EVIDENCE
0,18,"[['Bronchitis', 0.19171203430383882], ['Pneumo...",M,URTI,"['E_48', 'E_50', 'E_53', 'E_54_@_V_161', 'E_54...",E_91
1,21,"[['HIV (initial infection)', 0.518950056440760...",M,HIV (initial infection),"['E_9', 'E_27', 'E_50', 'E_51', 'E_53', 'E_54_...",E_50
2,19,"[['Bronchitis', 0.11278064619119596], ['Pneumo...",F,Pneumonia,"['E_53', 'E_54_@_V_179', 'E_54_@_V_192', 'E_55...",E_77
3,34,"[['URTI', 0.23859396799565236], ['Cluster head...",F,URTI,"['E_48', 'E_53', 'E_54_@_V_183', 'E_55_@_V_89'...",E_53
4,36,"[['URTI', 0.23677812769175735], ['Influenza', ...",M,URTI,"['E_49', 'E_50', 'E_53', 'E_54_@_V_183', 'E_55...",E_201
...,...,...,...,...,...,...
1025597,18,"[['Epiglottitis', 0.28156957795466475], ['HIV ...",M,Epiglottitis,"['E_33', 'E_53', 'E_54_@_V_179', 'E_54_@_V_192...",E_91
1025598,28,"[['Epiglottitis', 0.3703962237298842], ['Laryg...",F,Epiglottitis,"['E_53', 'E_54_@_V_192', 'E_55_@_V_20', 'E_55_...",E_91
1025599,0,"[['Epiglottitis', 0.13193905052537108], ['Crou...",F,Epiglottitis,"['E_33', 'E_53', 'E_54_@_V_192', 'E_55_@_V_20'...",E_194
1025600,26,"[['Epiglottitis', 0.3028258988138983], ['Acute...",F,Epiglottitis,"['E_53', 'E_54_@_V_179', 'E_54_@_V_192', 'E_55...",E_194


In [17]:
ds[ds.PATHOLOGY== 'Anemia'].count()

AGE                       50665
DIFFERENTIAL_DIAGNOSIS    50665
SEX                       50665
PATHOLOGY                 50665
EVIDENCES                 50665
INITIAL_EVIDENCE          50665
dtype: int64

In [19]:
# count of patients with all diseases
ds.PATHOLOGY.value_counts()

PATHOLOGY
URTI                                        64368
Viral pharyngitis                           61642
Anemia                                      50665
HIV (initial infection)                     29013
Localized edema                             27825
Anaphylaxis                                 27718
Pulmonary embolism                          27468
Influenza                                   26812
Bronchitis                                  26400
Allergic sinusitis                          26203
Acute dystonic reactions                    25982
GERD                                        25979
Acute otitis media                          25917
Pneumonia                                   25761
Panic attack                                25019
Acute laryngitis                            24129
Guillain-Barré syndrome                     22867
Pericarditis                                22785
Sarcoidosis                                 21285
Possible NSTEMI / STEMI                 

In [65]:
# sample a nex dataset from ds with 140 patient of 7 disease  and 20 each properly distributed for column SEX using stratified k fold
sample = ds.groupby('PATHOLOGY').apply(lambda x: x.sample(50)).reset_index(drop=True)


/var/folders/dg/ck0zj8w55sd57d00ccc4c1pc0000gn/T/ipykernel_31988/3220360419.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample = ds.groupby('PATHOLOGY').apply(lambda x: x.sample(50)).reset_index(drop=True)


In [68]:
diseases = ['Anemia', 'URTI', 'Pneumonia', 'HIV (initial infection)', 'Viral pharyngitis', 'Influenza', 'Tuberculosis']
updated_ds = sample[sample.PATHOLOGY.isin(diseases)]

In [69]:
sample.PATHOLOGY.value_counts()

PATHOLOGY
Acute COPD exacerbation / infection         50
Inguinal hernia                             50
Localized edema                             50
Myasthenia gravis                           50
Myocarditis                                 50
PSVT                                        50
Pancreatic neoplasm                         50
Panic attack                                50
Pericarditis                                50
Pneumonia                                   50
Possible NSTEMI / STEMI                     50
Pulmonary embolism                          50
Pulmonary neoplasm                          50
SLE                                         50
Sarcoidosis                                 50
Scombroid food poisoning                    50
Spontaneous pneumothorax                    50
Spontaneous rib fracture                    50
Stable angina                               50
Tuberculosis                                50
URTI                                        50
Uns

In [32]:
# iterate over the updated_ds and generate a report for each patient
for index, row in updated_ds.iterrows():
    evidences = row['EVIDENCES']

,AGE,DIFFERENTIAL_DIAGNOSIS,SEX,PATHOLOGY,EVIDENCES,INITIAL_EVIDENCE
160,36,"[['Anemia', 0.44587155073498674], ['HIV (initi...",M,Anemia,"['E_7', 'E_53', 'E_54_@_V_161', 'E_54_@_V_180'...",E_76
161,30,"[['Anemia', 0.6496640351985898], ['Cluster hea...",M,Anemia,"['E_7', 'E_53', 'E_54_@_V_180', 'E_54_@_V_182'...",E_53
162,45,"[['Anemia', 0.38161737656069056], ['HIV (initi...",M,Anemia,"['E_7', 'E_24', 'E_26', 'E_53', 'E_54_@_V_161'...",E_154
163,0,"[['Anemia', 0.1402393853581791], ['Atrial fibr...",F,Anemia,"['E_24', 'E_26', 'E_53', 'E_54_@_V_180', 'E_55...",E_89
164,7,"[['Anemia', 0.776039045795465], ['Scombroid fo...",F,Anemia,"['E_24', 'E_26', 'E_53', 'E_54_@_V_182', 'E_54...",E_82
...,...,...,...,...,...,...
955,56,"[['Viral pharyngitis', 0.16777641307094804], [...",M,Viral pharyngitis,"['E_41', 'E_45', 'E_48', 'E_49', 'E_53', 'E_54...",E_201
956,20,"[['Viral pharyngitis', 0.16597158147844399], [...",F,Viral pharyngitis,"['E_45', 'E_48', 'E_49', 'E_53', 'E_54_@_V_161...",E_45
957,77,"[['Viral pharyngitis', 0.41491517949822926], [...",M,Viral pharyngitis,"['E_41', 'E_45', 'E_48', 'E_49', 'E_53', 'E_54...",E_53
958,19,"[['Viral pharyngitis', 0.1981205785871585], ['...",M,Viral pharyngitis,"['E_41', 'E_45', 'E_49', 'E_53', 'E_54_@_V_161...",E_181


### Generate condition symptom and antecedents mapping

In [34]:
import json
conditions = json.load(open('release_conditions.json'))
evidences = json.load(open('release_evidences.json'))

In [46]:
path_condition_mapping = {}
unique_symptoms = []
unique_antecedents = []
for condition_key in conditions:
  name = conditions[condition_key]['condition_name']
  if name not in diseases:
    continue
  symptoms = list(conditions[condition_key]['symptoms'].keys())
  antecedents = list(conditions[condition_key]['antecedents'].keys())
  updated_symptoms = []
  updated_antecedents = []
  for symptom in symptoms:
    symptom_q = evidences[symptom]['question_en']
    updated_symptoms.append(symptom_q)
    if symptom_q not in unique_symptoms:
      unique_symptoms.append({
        'question': symptom_q,
        'name': evidences[symptom]['name']
      })
      
  for antecedent in antecedents:
    antecedent_q = evidences[antecedent]['question_en']
    updated_antecedents.append(antecedent_q)
    if antecedent_q not in unique_antecedents:
      unique_antecedents.append({
        'question': antecedent_q,
        'name': evidences[antecedent]['name']
      })
      
    path_condition_mapping[name] = {
      'name': name,
      'symptoms': updated_symptoms,
      'antecedents': updated_antecedents
    }

In [55]:
len(unique_symptoms)

121

In [52]:
def append_row(df, row):
    return pd.concat([
                df, 
                pd.DataFrame([row], columns=row.index)]
           ).reset_index(drop=True)

In [53]:
# Empty dataframe
symptom_ds = pd.DataFrame(columns=['name', 'symptom'])
failed_symptoms = []
for symptom in unique_symptoms:
  try:
    question = symptom['question']
    prompt = f"""You are given a doctor's question to patient  now your job to identify what symptom doctor trying to establish from the question, return me the symptom name in 2-4 words, example,
      Question: Do you have a cough?
      Cough
      Question: Do you have a fever?
      Fever
      Question: Do you have sore throat?
      Sore throat
      Question:{question}"""
    response = model.generate_content(prompt)
    row = pd.Series({
      'name': symptom['name'],
      'symptom': response.text
    })
    symptom_ds = append_row(symptom_ds, row)
    print("Generated response for symptom", row['symptom'])
  except Exception as e:
    print("Failed to generate response for symptom", symptom)
    print(e)
    failed_symptoms.append(symptom)
    continue
  

Generated response for symptom Fever
Generated response for symptom Fatigue
Generated response for symptom Muscle pain
Generated response for symptom Weight loss
Generated response for symptom Pain
Generated response for symptom Pain
Generated response for symptom Radiating pain
Generated response for symptom Pain
Generated response for symptom Pain onset
Generated response for symptom Pain intensity
Generated response for symptom Pain location
Generated response for symptom Sore throat
Generated response for symptom Swollen lymph nodes
Generated response for symptom Diarrhea
Generated response for symptom Location of affected region
Generated response for symptom Skin lesions
Generated response for symptom Rash
Generated response for symptom Rash pain intensity
Generated response for symptom Rash swelling
Generated response for symptom Itching
Generated response for symptom Lesion size
Generated response for symptom Skin peeling
Generated response for symptom Nausea or vomiting
Genera

In [56]:
temp_series = pd.Series({
  'name': 'E_181',
  'symptom': 'Runny/Congested nose'
})
symptom_ds = append_row(symptom_ds, temp_series)

In [58]:
symptom_ds.to_csv('symptoms.csv', index=False)

In [60]:
antecedent_ds = pd.DataFrame(columns=['name', 'antecedent'])
failed_antecedents = []
for antecedent in unique_antecedents:
  try:
    question = antecedent['question']
    prompt = f"""You are given a doctor's question to patient  now your job to identify what antecedent doctor trying to establish from the question, return me the antecedent name in 2-4 words, example,
      Question: Do you smoke cigarettes?
      Smoking
      Question: Do you have high blood pressure or do you take medications to treat high blood pressure?
      High blood pressure
        Question: Do you live with 4 or more people?
        Crowded living conditions
      Question:{question}"""
    response = model.generate_content(prompt)
    row = pd.Series({
      'name': antecedent['name'],
      'antecedent': response.text
    })
    antecedent_ds = append_row(antecedent_ds, row)
    print("Generated response for antecedent", row['antecedent'])
  except Exception as e:
    print("Failed to generate response for antecedent", antecedent)
    print(e)
    failed_antecedents.append(antecedent)
    continue

Generated response for antecedent Unprotected sex
Generated response for antecedent Intravenous drug use
Generated response for antecedent Sexually transmitted infection
Generated response for antecedent HIV status
Generated response for antecedent Travel
Generated response for antecedent Diet
Generated response for antecedent Anemia
Generated response for antecedent Anemia
Generated response for antecedent Pregnancy
Generated response for antecedent Chronic kidney failure
Generated response for antecedent Underweight
Generated response for antecedent Oral anticoagulants
Generated response for antecedent Travel history
Generated response for antecedent Daycare attendance
Failed to generate response for antecedent {'question': 'Do you live with 4 or more people?', 'name': 'E_48'}
429 Quota exceeded for quota metric 'Generate Content API requests per minute' and limit 'GenerateContent request limit per minute for a region' of service 'generativelanguage.googleapis.com' for consumer 'proj

In [61]:
antecedent_ds.to_csv('antecedents.csv', index=False)

### Explore the patients

In [72]:
updated_ds

,AGE,DIFFERENTIAL_DIAGNOSIS,SEX,PATHOLOGY,EVIDENCES,INITIAL_EVIDENCE
400,31,"[['Anemia', 0.16248459669351575], ['Atrial fib...",F,Anemia,"['E_7', 'E_24', 'E_26', 'E_53', 'E_54_@_V_182'...",E_145
401,44,"[['Anemia', 0.5243554749117972], ['Cluster hea...",F,Anemia,"['E_24', 'E_26', 'E_53', 'E_54_@_V_180', 'E_54...",E_145
402,26,"[['Anemia', 0.424119115524829], ['Cluster head...",F,Anemia,"['E_7', 'E_24', 'E_53', 'E_54_@_V_182', 'E_55_...",E_88
403,63,"[['Anemia', 0.5792667822640574], ['Cluster hea...",F,Anemia,"['E_7', 'E_24', 'E_26', 'E_53', 'E_54_@_V_180'...",E_88
404,103,"[['Anemia', 0.17163623347391554], ['Panic atta...",F,Anemia,"['E_7', 'E_24', 'E_53', 'E_54_@_V_161', 'E_54_...",E_53
...,...,...,...,...,...,...
2395,61,"[['Possible NSTEMI / STEMI', 0.333029896487044...",M,Viral pharyngitis,"['E_41', 'E_45', 'E_48', 'E_49', 'E_53', 'E_54...",E_53
2396,28,"[['Viral pharyngitis', 0.22702125813983617], [...",F,Viral pharyngitis,"['E_41', 'E_48', 'E_53', 'E_54_@_V_181', 'E_55...",E_201
2397,52,"[['Bronchitis', 0.337784629934469], ['Viral ph...",F,Viral pharyngitis,"['E_41', 'E_48', 'E_49', 'E_53', 'E_54_@_V_161...",E_201
2398,62,"[['Viral pharyngitis', 0.16777641307094804], [...",M,Viral pharyngitis,"['E_45', 'E_49', 'E_53', 'E_54_@_V_181', 'E_55...",E_181


In [123]:
symp_ds = pd.read_csv('symptoms.csv')
ant_ds = pd.read_csv('antecedents.csv')

In [124]:
symp_ds

,name,symptom
0,E_91,Fever
1,E_88,Fatigue
2,E_144,Muscle pain
3,E_162,Weight loss
4,E_55,Pain
...,...,...
116,E_201,Cough
117,E_66,Shortness of breath
118,E_94,Chills or shivers
119,E_144,Muscle pain


In [138]:
import ast
final_patient_ds = pd.DataFrame(columns=['age', 'sex', 'pathology', 'antecedents', 'symptoms'])
unique_antecedents_all = []
unique_symptoms_all = []
for patient in updated_ds.iterrows():
  patient_dict = {
    'age': patient[1]['AGE'],
    'sex': patient[1]['SEX'],
    'pathology': patient[1]['PATHOLOGY'],
    'antecedents': None,
    'symptoms': None
  }
  evidences_vals = ast.literal_eval(patient[1]['EVIDENCES'])
  symptoms_text = []
  antecedents_text = []
  # From format [evidence-name]_@_[evidence-value] extract the evidence name and value
  for evidence in evidences_vals:
    evidence_name  = None
    antecedent_name = None
    symptom_name = None
    if evidence in evidences:
      evidence_name  = evidence
    else:
      evidence_name = evidence.split('_@_')[0]
    evidence_details = evidences[evidence_name]
    if evidence_details["is_antecedent"]:
      antecedents_text.append(evidence_name)
    else:
      symptoms_text.append(evidence_name)
    
    if evidence_name not in unique_antecedents_all and evidence_details["is_antecedent"]:
        unique_antecedents_all.append(evidence_name)
    if evidence_name not in unique_symptoms_all and not evidence_details["is_antecedent"]:
        unique_symptoms_all.append(evidence_name)
  
  ants = set(antecedents_text)
  ants_text = []
  symps = set(symptoms_text)
  symps_text = []
  for ant in ants:
    antecedent_row = ant_ds[ant_ds['name'] == ant]
    if not antecedent_row.empty:
      ants_text.append(antecedent_row['antecedent'].values[0])
          
  for symp in symps:
    symptom_row = symp_ds[symp_ds['name'] == symp]
    if not symptom_row.empty:
      symps_text.append(symptom_row['symptom'].values[0])
          
  ants_text = list(set(ants_text))
  symps_text = list(set(symps_text))
  
  patient_dict['antecedents'] = ",".join(ants_text) if len(ants_text) > 0 else "None"
  patient_dict['symptoms'] = ",".join(symps_text) if len(symps_text) > 0 else "None"
      
  final_patient_ds = append_row(final_patient_ds, pd.Series(patient_dict))

In [139]:
final_patient_ds

,age,sex,pathology,antecedents,symptoms
0,31,F,Anemia,"Travel,Underweight,Anemia,Diet,Chronic kidney ...","Black stools,Radiating pain,Pain,Lightheadedne..."
1,44,F,Anemia,"Travel,Underweight,Pregnancy,Oral anticoagulan...","Black stools,Radiating pain,Pain,Lightheadedne..."
2,26,F,Anemia,"Travel,Underweight,Pregnancy,Oral anticoagulan...","Black stools,Radiating pain,Pain,Pain location..."
3,63,F,Anemia,"Travel,Pregnancy,Oral anticoagulants,Anemia,Di...","Radiating pain,Pain,Pain location,Fatigue,Pain..."
4,103,F,Anemia,"Travel,Underweight,Pregnancy,Oral anticoagulan...","Radiating pain,Pain,Lightheadedness,Pain locat..."
...,...,...,...,...,...
345,61,M,Viral pharyngitis,"Travel,Smoking,Contact with infected person,Da...","Radiating pain,Pain,Pain location,Coughing up ..."
346,28,F,Viral pharyngitis,"Travel,Smoking,Contact with infected person,Cr...","Radiating pain,Pain,Pain location,Pain intensi..."
347,52,F,Viral pharyngitis,"Travel,Contact with infected person,Daycare at...","Radiating pain,Pain,Pain location,Pain intensi..."
348,62,M,Viral pharyngitis,"Daycare attendance,Travel,Immunosuppression","Radiating pain,Pain,Pain location,Coughing up ..."


In [140]:
final_patient_ds.to_csv('patients.csv', index=False)

In [121]:
for u_s in unique_symptoms_all:
  found = False
  for u_s_ in unique_symptoms:
    if u_s_['name'] == u_s:
      found = True
  if not found:
    print(u_s)

In [122]:
for u_a in unique_antecedents_all:
  found = False
  for u_a_ in unique_antecedents:
    if u_a_['name'] == u_a:
      found = True
  if not found:
    print(u_a)

### Generate the synthetic patient doctor convo summary

In [141]:
generation_config_1 = {
  "candidate_count": 1,
  "max_output_tokens": 256,
  "temperature": 0.8
}
generation_model = genai.GenerativeModel('gemini-pro', safety_settings=safety_settings, generation_config=generation_config_1)

In [163]:
from openai import OpenAI
        
client = OpenAI(
  api_key='',
  base_url="https://dbc-71f6065e-3393.cloud.databricks.com/serving-endpoints"
)

In [170]:
master_ds = pd.DataFrame(columns=['age', 'sex', 'pathology', 'antecedents', 'symptoms', 'summary'])
count = 0
for index, row in final_patient_ds.iterrows():
  age = row['age']
  sex = row['sex']
  pathology = row['pathology']
  symptoms = row['symptoms']
  antecedents = row['antecedents']
  messages = [
  {
    "role": "system",
    "content": "You are an expert assistant helping a doctor to simulate a summary of the patient's condition and the conversation you had with the patient, the summary should be in 100-500 words, try to be as realistic as possible, write it as simple summary no need to add points in third person POV."
  },
  {
    "role": "user",
    "content": "You are a doctor and you are seeing a patient for first or some time now, you have the following information about the patient,  patient is 31 years of gender Female and has been diagnosed with Anemia and has the following symptoms Black stools,Radiating pain,Pain,Lightheadedness,Pain location,Shortness of breath,Dizziness or lightheadedness,Fatigue,Pain intensity,Blood in stool,Pain onset,Heavy menstrual bleeding, and has the following antecedents Travel,Underweight,Anemia,Diet,Chronic kidney failure"
  },{
    "role": "assistant",
    "content": """The patient, a 31-year-old female, visited the clinic due to a range of symptoms related to anemia. She reported feeling increasingly fatigued and dizzy, with episodes of black stools and blood in her stool, suggesting potential gastrointestinal bleeding. Additionally, she experiences severe and radiating pain along with heavy menstrual bleeding, which likely exacerbates her anemia.Her medical history includes frequent travel, being underweight, and dietary challenges. She also has a family history of chronic kidney failure, though she has not been diagnosed with this herself. During our consultation, we discussed her symptoms and medical history in detail. Given her symptoms and history, I ordered laboratory tests including a complete blood count and an assessment for possible gastrointestinal bleeding. We discussed starting iron supplementation immediately and possibly other treatments depending on her test results. We also talked about improving her diet with the help of a nutritionist. The patient was engaged in the discussion and expressed her concerns about her symptoms and their impact on her life. We scheduled a follow-up appointment to review her test results and adjust her treatment plan as needed."""
    }, {
    "role": "user",
    "content": f"""patient is {age} years of gender {"Male" if sex == 'M' else 'Female'} and has been diagnosed with {pathology} and has the following symptoms {symptoms}, and has the following antecedents {antecedents}"""
    }
  ]
  
  try:
    chat_completion = client.chat.completions.create(
    messages=messages,
    model="databricks-meta-llama-3-70b-instruct",
    max_tokens=512
    )
    content = chat_completion.choices[0].message.content
    row = pd.Series({
        'age': age,
        'sex': sex,
        'pathology': pathology,
        'antecedents': antecedents,
        'symptoms': symptoms,
        'summary': content
      })
    master_ds = append_row(master_ds, row)
    count += 1
    print("count", count)
  except Exception as e:
    print("Failed to generate response for patient", row)
  

count 1
count 2
count 3
count 4
count 5
count 6
count 7
count 8
count 9
count 10
count 11
count 12
count 13
count 14
count 15
count 16
count 17
count 18
count 19
count 20
count 21
count 22
count 23
count 24
count 25
count 26
count 27
count 28
count 29
count 30
count 31
count 32
count 33
count 34
count 35
count 36
count 37
count 38
count 39
count 40
count 41
count 42
count 43
count 44
count 45
count 46
count 47
count 48
count 49
count 50
count 51
count 52
count 53
count 54
count 55
count 56
count 57
count 58
count 59
count 60
count 61
count 62
count 63
count 64
count 65
count 66
count 67
count 68
count 69
count 70
count 71
count 72
count 73
count 74
count 75
count 76
count 77
count 78
count 79
count 80
count 81
count 82
count 83
count 84
count 85
count 86
count 87
count 88
count 89
count 90
count 91
count 92
count 93
count 94
count 95
count 96
count 97
count 98
count 99
count 100
count 101
count 102
count 103
count 104
count 105
count 106
count 107
count 108
count 109
count 110
count 11

In [172]:
master_ds.to_csv('master_ds.csv', index=False)